In [ ]:
#Problem 1
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as no
sns.set()
%matplotlib notebook

#new nba data function that gets shooting scores from example: https://stats.nba.com/player/201939/shooting/?Season=2016-17&SeasonType=Regular%20Season&sort=FG3M&dir=1

def get_nba_data_1(endpt, params, return_url=False):

    ## endpt: https://github.com/seemethere/nba_py/wiki/stats.nba.com-Endpoint-Documentation
    ## params: dictionary of parameters: i.e., {'LeagueID':'00'}
    
    from pandas import DataFrame
    from urllib.parse import urlencode
    import json
    
    useragent = "\"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9\""
    dataurl = "\"" + "http://stats.nba.com/stats/" + endpt + "?" + urlencode(params) + "\""
    
    # for debugging: just return the url
    if return_url:
        return(dataurl)
    
    jsonstr = !wget -q -O - --user-agent={useragent} {dataurl}
    
    data = json.loads(jsonstr[0])
    
    h = data['resultSets'][1]['headers']
    d = data['resultSets'][1]['rowSet']
    
    return(DataFrame(d, columns=h))

params = {'DateFrom':'',
          'DateTo':'',
          'GameSegment':'',
          'LastNGames':'0',
          'LeagueID':'00',
          'Location':'',
          'MeasureType':'Base',
          'Month':'0',
          'OpponentTeamID':'0',
          'Outcome':'',
          'PORound':'0',
          'PaceAdjust':'N',
          'PerMode':'Totals',
          'Period':'0',
          'PlayerID':'201939', #Corresponds to Steph Curry
          'PlusMinus':'N',
          'Rank':'N',
          'Season':'2016-17',
          'SeasonSegment':'',
          'SeasonType':'Regular Season',
          'ShotClockRange':'',
          'VsConference':'',
          'VsDivision':'',
         }



endpt= 'playerdashboardbyshootingsplits'

shooting = get_nba_data_1(endpt,params,return_url=False)

In [ ]:
shooting

In [ ]:
#Problem 2

from ipywidgets import interact, FloatSlider, Dropdown, Button

#I needed this function to curate and populate my dropdown lists

def get_nba_data(endpt, params, return_url=False):

    ## endpt: https://github.com/seemethere/nba_py/wiki/stats.nba.com-Endpoint-Documentation
    ## params: dictionary of parameters: i.e., {'LeagueID':'00'}
    
    from pandas import DataFrame
    from urllib.parse import urlencode
    import json
    
    useragent = "\"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9\""
    dataurl = "\"" + "http://stats.nba.com/stats/" + endpt + "?" + urlencode(params) + "\""
    
    # for debugging: just return the url
    if return_url:
        return(dataurl)
    
    jsonstr = !wget -q -O - --user-agent={useragent} {dataurl}
    
    data = json.loads(jsonstr[0])
    
    h = data['resultSets'][0]['headers']
    d = data['resultSets'][0]['rowSet']
    
    return(DataFrame(d, columns=h))


In [ ]:
#Populating the names of Gold State Warrior Players
params = {'LeagueID':'00',
          'Season':'2016-17',
          'TeamID':'1610612744' #corresppnds to Golden State Warriors
         }


players = get_nba_data('commonteamroster', params)
players
#creating a dictionary with player names and IDs.
player_name = dict(zip(list(players.PLAYER), list(players.PLAYER_ID)))
player_name 



In [ ]:
#Problem 2 and 3 since the buttons are created and interactive.

names = list(player_name.keys())
names.sort()
menu = {
    'player name':names ,
    'Season Type':[ 'Regular Season', 'Playoffs'],
}



player = Dropdown(options=menu['player name'], value=menu['player name'][0])
season = Dropdown(options=menu['Season Type'], value=menu['Season Type'][0])

download = Button(description='Display Shot Stats!', icon='check')


def display_shot_stats(change):
    params = {'DateFrom':'',
          'DateTo':'',
          'GameSegment':'',
          'LastNGames':'0',
          'LeagueID':'00',
          'Location':'',
          'MeasureType':'Base',
          'Month':'0',
          'OpponentTeamID':'0',
          'Outcome':'',
          'PORound':'0',
          'PaceAdjust':'N',
          'PerMode':'Totals',
          'Period':'0',
          'PlayerID':player_name[player.value],
          'PlusMinus':'N',
          'Rank':'N',
          'Season':'2016-17',
          'SeasonSegment':'',
          'SeasonType':season.value,
          'ShotClockRange':'',
          'VsConference':'',
          'VsDivision':'',
         }
    endpt= 'playerdashboardbyshootingsplits'
    df=get_nba_data_1(endpt,params,return_url=False)
    df["Player_Name"] =player.value
    display(df)
    return(df)
    

    
download.on_click(display_shot_stats)
display(player, season, download)

In [ ]:
# Problem 4 Getting all Golden State Warriors Data
GSW_playoffs_data=pd.DataFrame()
for i in names:
    print(i)
    params = {'DateFrom':'',
          'DateTo':'',
          'GameSegment':'',
          'LastNGames':'0',
          'LeagueID':'00',
          'Location':'',
          'MeasureType':'Base',
          'Month':'0',
          'OpponentTeamID':'0',
          'Outcome':'',
          'PORound':'0',
          'PaceAdjust':'N',
          'PerMode':'Totals',
          'Period':'0',
          'PlayerID':player_name[i],
          'PlusMinus':'N',
          'Rank':'N',
          'Season':'2016-17',
          'SeasonSegment':'',
          'SeasonType':'Playoffs',
          'ShotClockRange':'',
          'VsConference':'',
          'VsDivision':'',
         }
    endpt= 'playerdashboardbyshootingsplits'
    temp=get_nba_data_1(endpt,params,return_url=False)
    temp["Player_Name"] =i
    GSW_playoffs_data=GSW_playoffs_data.append(temp,ignore_index=False)
    

In [ ]:
GSW_playoffs_data



In [ ]:
# Creating overall metrics by Player of the whole team by collapsing shot range variable
#source : https://stats.nba.com/player/201939/shooting/?Season=2016-17&SeasonType=Regular%20Season&sort=FG3M&dir=1
#My Initial Data set has the shooting scores stratified by Shot Distace at 5 ft intervals. It is the second table on the source website
#What I am trying to do here is create Overall metrics that are on the top of my source website

GSW_playoffs_data2 = (GSW_playoffs_data.groupby("Player_Name") # combine step
               .apply(lambda x:pd.Series({'Field_Goals_Made':x.FGM.sum(),
                                          'Field_Goals_Attempted':x.FGA.sum(),
                                          'Field_Goals_Made_3pt':x.FG3M.sum(),
                                          'Field_Goals_Attempted_3pt':x.FG3A.sum(),
                                          'Blocked_Field_Goal_Attempts':x.BLKA.sum(),
                                          'Player':x.Player_Name.max()
                                      
                                         }))
                      .assign(FGP = lambda x:x.Field_Goals_Made / x.Field_Goals_Attempted) #Field_Goals_percentage
                      .assign(FGP3pt = lambda x:x.Field_Goals_Made_3pt / x.Field_Goals_Attempted_3pt)#Field_Goals_percentage_3pt
                      .assign(EFGP = lambda x:(x.Field_Goals_Made + (0.5 *x.Field_Goals_Made_3pt)/ x.Field_Goals_Attempted))#Effective_Field_Goal_Percentage

                                         )# similar to mutate() in dplyr

GSW_playoffs_data2 

In [ ]:
# I am not sure how to make the labels vertical. Also Not sure whether I have enough data to Plot.
import seaborn as sns

%matplotlib inline
sns.set()
sns.factorplot(x="Player", y="EFGP", data=GSW_playoffs_data2 )
